In [2]:
# %load init.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('../data/complaints.csv')

In [3]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
4,2019-02-08,Vehicle loan or lease,Lease,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,HYUNDAI CAPITAL AMERICA,FL,33161,NaN,Consent not provided,Web,2019-02-08,Closed with non-monetary relief,Yes,NaN,3146310


In [33]:
df['Date'] = pd.to_datetime(df['Date received'],infer_datetime_format=True)
df = df.rename(columns={'Complaint ID': 'All_Issues'})
df = df.rename(columns={'Timely response?': 'Responsive'})
dfg = df.groupby(['Responsive','Company']).count()[['All_Issues']].sort_values('All_Issues', ascending=False)
dfg = dfg.reset_index()
dfg 


,Responsive,Company,All_Issues
0,Yes,"EQUIFAX, INC.",151153
1,Yes,Experian Information Solutions Inc.,135717
2,Yes,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",132192
3,Yes,"BANK OF AMERICA, NATIONAL ASSOCIATION",87090
4,Yes,WELLS FARGO & COMPANY,72928
...,...,...,...
8260,Yes,"Keaveney Legal Group, LLC",1
8261,No,Meridian Buyer’s Group,1
8262,Yes,"Keith A. Ringelspaugh, P.A.",1
8263,Yes,Kelly Harvey P.C.,1


In [37]:
dfpivot = dfg.pivot(index='Responsive', columns='Company', values='All_Issues')
dfpivot = dfpivot.reset_index()
dfpivot = dfpivot.fillna(0)
dfpivot

Company,Responsive,"(Former)Shapiro, Swertfeger & Hasty, LLP","1 STOP MONEY CENTERS, LLC",10 Cent Title Pawn Inc,1ST 2ND MORTGAGE CO. OF NJ INC,"1ST ALLIANCE LENDING, LLC","1ST FINANCIAL, INC.",1ST PREFERENCE MORTGAGE CORP,"1ST RESULTS BILLINGS & COLLECTIONS, INC.","1st Capital Finance of South Carolina, Inc.",...,eToro USA LLC,"i3 Lending, Inc","iCollect.com, Corp.",iFreedom Direct Corporation,"iMove Mortgage, LLC","iQuantified Management Services, LLC",iServe Trust,snw investments,"xCoins, Inc.","Lippman Recupero, LLC"
0,No,11.0,0.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,2.0,0.0,3.0,2.0,0.0,0.0,1.0
1,Yes,0.0,5.0,2.0,1.0,16.0,6.0,2.0,3.0,3.0,...,1.0,1.0,0.0,20.0,1.0,7.0,12.0,1.0,1.0,5.0


In [40]:
#dfpivot.loc[:,dfg['Company']].plot.bar(stacked=True)

ValueError: Image size of 616x124026 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 432x288 with 1 Axes>